# 4. GRACE satellite application
## Monitoring the evolution of the Antarctic Ice Sheet

## Introduction

The Antarctic Ice Sheet (AIS) is the largest block of ice on Earth. It is unique by its thickness, which averages 2 160 meters, and its surface of approximately 14 million square kilometers (an area larger than Europe), making it the biggest freshwater reservoir in the world. The ice sheet on the continent extends over the ocean and becomes ice shelf. This mass of ice plays a crucial role in the regulation of the Earth Climate, thanks to the relfective properties of the white ice covered areas surrounded by the dark sea surface, or the cold ocean surrounding the continent driving the oceaninc circulation around the globe. Its dynamics and evolution is a topic of significant concern for the scientific community.

Antarctica is particularly sensible to climate change, although there is a large climate variability over the region. The continent lies at the crossroads of numerous forcings : the ozone hole, disturbances caused by tropical storms and moonsoons, and of course the increases of greenhous gases in the atmosphere. This makes the Antartic Ice Sheet highly dynamic: climate records from the Antartic Peninsula show that  temperatures in this region rose by around 3.2°C in the second half of the 20th century, which is more than three times the global average. West Antartica has been melting at an accelerating rate, while the data on East Antarctica is less clear. In the end, as many unknowns remain about the processes governing the region and its potential feedback responses to climate change, there are still many uncertainties in the future contribution of the melting of the AIS to sea-level rise. According to the Fifth Assessment Report (AR5) from the Intergovernmental Panel on Climate Change (IPCC), even in the most extreme emissions scenario where surface mass gains in the East Antarctic Ice Sheet (EAIS) compensate for projected ice mass loss, the Antarctic Ice Sheet (AIS) is projected to contribute only a modest -0.08 to 0.14 meters to global sea level rise by the year 2100 ([Church et al., 2013](https://www.ipcc.ch/site/assets/uploads/2018/02/WG1AR5_Chapter13_FINAL.pdf)). On the other hand, in a high-temperature scenario, the expert assessment conducted by [Bamber et al. (2019)](https://doi.org/10.1073/pnas.1817205116) estimated a likely range of 0.03 to 0.46 meters from the West Antarctic Ice Sheet (WAIS) and a range of -0.04 to 0.11 meters from the East Antarctic Ice Sheet (EAIS) for the year 2100. These estimates are expected to increase to a range of 0.07 to 2.28 meters (WAIS) and -0.14 to 0.51 meters (EAIS) by the year 2300.

Monitoring the evolution of the AIS is therefore crucial to understand climate change and forecasts its future impacts across the globe. Measuring the mass of the Antarctic Ice sheet is however still a complex scientific task, and many methods exist including satellite altimetry, gravimetry, ice core analysis, ground-based GPS, airborne surveys, ice sheet models, etc.

In this section, we are going to use the [Gravimetric mass balance data for the Antarctic and Greenland ice sheets from 2003 to 2020 derived from satellite observations](https://www.wekeo.eu/data?view=dataset&dataset=EO%3AECMWF%3ADAT%3ASATELLITE_ICE_SHEET_MASS_BALANCE) product. This dataset offers a comprehensive collection of monthly time series data depicting alterations in mass for the drainage basins of both the Antarctic and Greenland Ice Sheets. These data are sourced from NASA's Gravity Recovery and Climate Experiment (GRACE) and GRACE Follow-On (GRACE-FO) missions, which have been in operation since 2002 to the present day. These observations align with the targets established by the Global Climate Observing System for the Essential Climate Variable of Ice Sheets Mass Change. The GRACE twin satellites, through their monitoring of fluctuations in Earth's gravitational field, have allowed us to deduce changes in mass across the ice sheets using a regional integration approach. This dataset includes **time series data** illustrating **mass changes** for each individual ice sheet as well as for each specific drainage basin.

| Parameter | Value |
| :---: | :---|
| **Variables used** | AntIS_xxx, AntIS_xxx_er,GrIS_xxx, GrIS_xxx_er |
| **Product identifier** |[EO:ECMWF:DAT:SATELLITE_ICE_SHEET_MASS_BALANCE](https://www.wekeo.eu/data?view=dataset&dataset=EO%3AECMWF%3ADAT%3ASATELLITE_ICE_SHEET_MASS_BALANCE) |
| **Temporal resolution** | Monthly averaged values |
| **Temporal coverage** | December 2002 to December 2022 |


<center>
    <img src="Data/GRACE/GRACE_product.png" width="300">
</center>

You can also visit the [Copernicus page](https://cds.climate.copernicus.eu/cdsapp#!/dataset/satellite-ice-sheet-mass-balance?tab=overview) dedicated to the product to see more detail about the product and all the different variables that are available.

In [ ]:
!pip install hda xarray geopandas fiona netCDF4 h5netcdf scipy

In [ ]:
# Modules system
import warnings
warnings.filterwarnings('ignore')
import getpass
import os
from pathlib import Path

# Modules related to data retrieving
from hda import Client
import json

# Modules related to plot and EO data manipulation
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import geopandas as gpd
from shapely.geometry import box, Polygon, MultiPolygon
from fiona.crs import from_epsg
from rasterio.mask import mask

In [ ]:
# Try netCDF4 engine
try:
    dsi = xr.open_dataset('Data/GRACE/C3S_GMB_GRACE_vers4.nc', engine='netcdf4')
    print("Success with netCDF4 engine")
except Exception as e:
    print("netCDF4 failed:", e)

# Try h5netcdf engine  
try:
    dsi = xr.open_dataset('Data/GRACE/C3S_GMB_GRACE_vers4.nc', engine='h5netcdf')
    print("Success with h5netcdf engine")
except Exception as e:
    print("h5netcdf failed:", e)

In [ ]:
dsi = xr.open_dataset('Data/GRACE/C3S_GMB_GRACE_vers4.nc')
dsi

In [ ]:
# re-generating the dataset at a convenient format
ds = xr.Dataset(coords=dict(time=dsi.time),attrs=dsi.attrs)

In [ ]:
for basin_nb in range(1,9):
    name = "GrIS_%d" % basin_nb
    ds[name] = dsi[name]
    ds[name+"_er"] = dsi[name+"_er"]

ds['GrIS_total'] = dsi['GrIS_total']
ds['GrIS_total_er'] = dsi['GrIS_total_er']

# resampling to have a regular time scale and filling holes with interpolation
ds = ds.resample(time='1M').mean().interpolate_na(dim='time')

ds['GrIS_total_er']

In [ ]:
# Check the date times included in the file
ds['time']
# If you know the exact date
ds.sel(time='2014-06')
# If you don't know the exact date and want to take the nearest one
ds.sel(time='2014-06-05', method='nearest')
# If you want to select a date from its index
ds.isel(time=[3])
# If you want to select a range of dates
# Here, let's for example select the first semester of 2015

ds.sel(time=slice('2015-01-01','2015-06-30'))

In [ ]:
fig,ax=plt.subplots(figsize=(12,6))

dates = ds.time.astype('datetime64')
vals = ds.GrIS_total.data
er = ds.GrIS_total_er.data

ax.plot(dates,vals,color='k')
ax.fill_between(dates,vals-er,vals+er,facecolor='k',alpha=0.5)
ax.grid(True)
ax.set_ylabel('Gt')
ax.set_xlabel('year')
ax.set_title('Greenland Ice sheet anomaly over 2002-2022')

As expected, we see that the total mass of the Greenland Ice Sheet has been decreasing for the past twenty years, and that there is also a seasonal pattern in the ice mass change. In the next sections we will look at the phenomenon in more detail. For now we can see in the product that there are Ice Mass Balance time series for the whole region in Greenland.

Before looking at the changes in these regions, let's smooth the seasonal Ice Mass variations by performing a yearly resampling of the data. We will do that only over the full years of data, so from 2003 to 2022.

In [ ]:
ds_y = ds.resample(time='1Y').mean().sel(time=slice('2003','2022'))
ds_y

In [ ]:
# cumulative change

fig, ax = plt.subplots()

dates = ds_y.time.astype('datetime64')

vals_Greenland = ds_y.GrIS_total.data - ds_y.GrIS_total.isel(time=[0]).item()
er_p = ds_y.GrIS_total_er.data

ax.plot(dates, vals_Greenland, color='y', label='Greenland')
ax.fill_between(dates, vals_Greenland-er_p, vals_Greenland+er_p, facecolor='y', alpha=0.5)

ax.set_title("Total Ice Mass evolution since 2003")
ax.set_ylabel('Gt')
ax.legend()
ax.grid(True)

## 2. Ice Sheet Mass Change over the Greenland's drainage basins

Our dataset provides timeseries for 8 different Greenland drainage basins, but how are they distributed over the continent ? Let's have another look at the **initial dataset**, it has a variable related to Greenland that is not an ice mass change timeseries. It is the variable `GrBasin`. Let's quickly display the values. Remember that you can always check the product's [documentation](https://cds.climate.copernicus.eu/cdsapp#!/dataset/satellite-ice-sheet-mass-balance?tab=doc) to know more about the variables.

In [ ]:
dsi['GrISBasin'].data

### 2.1. Computing the polygons

The `GrISBasin` is indeed not a time series, it contains geographic information. It defines the contours of each Greenland drainage basin (as defined by [Zwally et al](https://earth.gsfc.nasa.gov/cryo/data/polar-altimetry/antarctic-and-greenland-drainage-systems)) : each line in the table gives a **latitude**, a **longitude**, and the corresponding **basin number**. In the next cell we derive the 8 polygons from the coordinates given in the variable. This may take some time to execute !

In [ ]:
dsi['GrISBasin'].data.shape

# creating polygons from coordinates:

# creating 8 empty list
basin_list = [[]for _ in range(8)]
for i in range(dsi['GrISBasin'].data.shape[0]):
    # filling each list with the corresponding basin coordinates
    basin_list[int(dsi['GrISBasin'].data[i,2]-1)].append(list(np.flip(dsi['GrISBasin'].data[i,:][0:2])))

In [ ]:
# computing polygons with geopandas
polygon_geom = []
for basin in range(len(basin_list)):
        polygon_geom.append(Polygon(basin_list[basin]))

## You might need this in Antarctic Ice Sheet Mass Balance changes
# polygon_geom = []
# for basin in range(len(basin_list)):
#     if basin == 16: 
#         # basin N°17 crosses the -180°/180° Longitude line.
#         # so this one will be split in two to have a coherent visualisation
#         poly1 = Polygon(basin_list[basin][39437:])
#         poly2 = Polygon(basin_list[basin][:39437])

#         multi_polygon = MultiPolygon([poly1, poly2])
#         polygon_geom.append(multi_polygon)
#     else:
#         polygon_geom.append(Polygon(basin_list[basin]))


gdf = gpd.GeoDataFrame(index=range(1,9), crs='epsg:3857', geometry=polygon_geom)
gdf['basin_nb'] = range(1,9)

# coordinates of polygon representative points for annotations
gdf['coords'] = gdf['geometry'].apply(lambda x: x.representative_point().coords[:])
gdf['coords'] = [coords[0] for coords in gdf['coords']]


In [ ]:
# the ".head()" method only displays the first 5 elements of the table, for better readability.
gdf.head()

In [ ]:
# delimit the area of interest around Greenland

LAT_MIN = 58
LAT_MAX = 85
LON_MIN = -90
LON_MAX = 0
LAT_CENTRAL = 70
LON_CENTRAL = -45

In [ ]:
# Define the caracteristics of the plot
f = plt.figure(figsize=(7, 7))                                                    # create a figure and define its size
ax = plt.axes(projection=ccrs.NearsidePerspective(central_longitude=LON_CENTRAL, central_latitude=LAT_CENTRAL))

## This command could be used to plot the Antarctic map
# ax = plt.axes(projection=ccrs.SouthPolarStereo())    # create an ax and select the projection of the map

ax.coastlines()                                                                    # add the coastlines
gl = ax.gridlines(draw_labels=True)                        # add the longitude / latitude lines
gl.right_labels = False                                                            # remove latitude labels on the right
gl.top_labels = False                                                              # remove longitude labels on the top
ax.add_feature(cfeature.LAND, zorder=1, edgecolor='k')                             # add land mask
ax.set_extent([LON_MIN, LON_MAX, LAT_MIN, LAT_MAX],crs=ccrs.PlateCarree())         # define the extent of the map [lon_min,lon_max,lat_min,lat_max]

# Plot the sea total precipitation, set the min/max values of the colorbar and the colormap to use
im = gdf.plot(ax=ax, transform=ccrs.PlateCarree(),  edgecolor='b', facecolor='w', linewidth=0.5)
# Add the titles and legend
ax.set_title("First look at the Greenland Basins GeoDataFrame")

for idx, row in gdf.iterrows():
    plt.annotate(text=row['basin_nb'], xy=row['coords'],
                 horizontalalignment='center', transform=ccrs.PlateCarree(), color='grey', fontsize=8)
    
# Save figure
# plt.savefig('Greenland_GDF.png')

### 2.2. Calculating trends and visualising Mass Change rate

The GRACE and GRACE-FO gravimetric products are particulary useful to calculate mass changes rates over long periods. We are going to perform linear regressions of the mass changes for each basins to see the spatial distribution of the ice mass change rates over the continent.

In [ ]:
# trend calculation:

# to ease trend calculation: manually specifying years
time = np.arange(2003,2023)
vals = ds_y['GrIS_6'].data ## take Basin 6 as an example

slope,intercept = np.polyfit(time.reshape(-1,1).flatten(),vals,1)
slope

In [ ]:
fig,ax = plt.subplots(figsize=(10,8))
ax.plot(time,vals,color='k')

# plot trend
ax.plot(time,slope*time+intercept,color='r',linestyle=':')
ax.grid(True)
ax.set_xticks([2000,2005,2010,2015,2020,2025])

ax.set_ylabel("Gt")
ax.set_xlabel("time")

ax.set_title("Example: Basin N°6 \nMass change rate: %.1f $Gt.y^{-1}$" %slope)

In [ ]:
# Now calculate for each basin
slope_list = []

for basin_label in range(1,9):
    vals = ds_y["GrIS_%d" % basin_label].data
    slope,intercept = np.polyfit(time.reshape(-1,1).flatten(),vals,1)
    slope_list.append(slope)

gdf['yearly_mass_change_rate'] = slope_list

In [ ]:
gdf.head()

In [ ]:
# Define the caracteristics of the plot
f = plt.figure(figsize=(10, 10))                                                    # create a figure and define its size
## For Antarctic 
## ax = plt.axes(projection=ccrs.SouthPolarStereo())                                       # create an ax and select the projection of the map
ax = plt.axes(projection=ccrs.NearsidePerspective(central_longitude=LON_CENTRAL, central_latitude=LAT_CENTRAL))
ax.coastlines()                                                                    # add the coastlines
gl = ax.gridlines(draw_labels=True)                        # add the longitude / latitude lines
gl.right_labels = False                                                            # remove latitude labels on the right
gl.top_labels = False                                                              # remove longitude labels on the top
ax.add_feature(cfeature.LAND, zorder=1, edgecolor='k')                             # add land mask
ax.set_extent([LON_MIN, LON_MAX, LAT_MIN, LAT_MAX],crs=ccrs.PlateCarree())         # define the extent of the map [lon_min,lon_max,lat_min,lat_max]

# Plot the sea total precipitation, set the min/max values of the colorbar and the colormap to use
im = gdf.plot(ax=ax, column='yearly_mass_change_rate', transform=ccrs.PlateCarree(), cmap='bwr_r', legend=True, vmin=-50, vmax=50, edgecolor='white', linewidth=0.5)
# Add the titles and legend
ax.set_title("Yearly Rate of Mass Change of the Greenland Ice Shelf\nin $Gt.y^{-1}$")

for idx, row in gdf.iterrows():
    plt.annotate(text=row['basin_nb'], xy=row['coords'],
                 horizontalalignment='center', transform=ccrs.PlateCarree(), color='grey', fontsize=8)
    
# Save figure
# plt.savefig('GMB_rate.png')

### 2.3. Analysing change rate acceleration

It is now clear that the drainage basins have very verying dynamics : most of West Greenland suffers from very fast ice loss, whereas Basin 2 shows basins that are actually no obviously trend over ice.

To deepen the analysis, we are now going to derive the timeseries in order to look at the trends followed by the ice mass change rates themselves. We can refer to those trends as the mass change acceleration.

In [ ]:
# same analysis, but with the derivative !

time = np.arange(2003, 2023)
slope_list = []

for basin_label in range(1,9):
    vals = np.gradient(ds_y["GrIS_%d" % basin_label].data)
    slope,intercept = np.polyfit(time.reshape(-1,1).flatten(),vals,1)
    slope_list.append(slope)

gdf['yearly_mass_change_acc'] = slope_list
gdf.head()

In [ ]:
# Define the caracteristics of the plot
f = plt.figure(figsize=(10, 10))                                                    # create a figure and define its size
## For Antarctic 
# ax = plt.axes(projection=ccrs.SouthPolarStereo())                                       # create an ax and select the projection of the map
ax = plt.axes(projection=ccrs.NearsidePerspective(central_longitude=LON_CENTRAL, central_latitude=LAT_CENTRAL))
ax.coastlines()                                                                    # add the coastlines
gl = ax.gridlines(draw_labels=True)                        # add the longitude / latitude lines
gl.right_labels = False                                                            # remove latitude labels on the right
gl.top_labels = False                                                              # remove longitude labels on the top
ax.add_feature(cfeature.LAND, zorder=1, edgecolor='k')                             # add land mask
ax.set_extent([LON_MIN, LON_MAX, LAT_MIN, LAT_MAX],crs=ccrs.PlateCarree())         # define the extent of the map [lon_min,lon_max,lat_min,lat_max]

# Plot the sea total precipitation, set the min/max values of the colorbar and the colormap to use
im = gdf.plot(ax=ax, column='yearly_mass_change_acc', transform=ccrs.PlateCarree(), cmap='seismic_r', legend=True, vmin=-5, vmax=5, edgecolor='white', linewidth=0.5)
# Add the titles and legend
ax.set_title("Yearly Acceleration of Mass Change of the Greenland Ice Shelf\nin $Gt.y^{-2}$")

for idx, row in gdf.iterrows():
    plt.annotate(text=row['basin_nb'], xy=row['coords'],
                 horizontalalignment='center', transform=ccrs.PlateCarree(), color='grey', fontsize=8)
    
# Save figure
# plt.savefig('GMB_acc.png')

We can see that the spatial variability of the ice mass change rates and the ice mass change accelerations are very different. For many of the basins that are loss ice mass (ex: basins N°8, 7, 1 ), the pace is actually slowing down over the years. Therefore, assuming the ice change rate keeps following a near-linear trend, there is a possiblity that some time in the future the ice change rate becomes negative and the drainage basins starts losing ice.

On the other hand, the basins in East Greenland that are already losing ice seem to follow an accelerating trend (ex: basins N°3), showing that East Greenland is definitely the most fragile side of the Greenland ice Sheet.

### 2.4. Making an animation

Now that we have learnt how to perform plots of the Greenland Ice Sheet, we can conclude this section by making an animation of its evolution.

In [ ]:
# preparing the geodataframe
ds_2y = ds_y.resample(time='2Y').mean()
ds_2y.time.data

In [ ]:
for c,date in enumerate(ds_2y.time.data):
    gdf[str(date.astype("datetime64[Y]").astype(int)+1970)] = [ds_2y['GrIS_%d' % basin_nb].isel(time=[c]).item() - ds_2y['GrIS_%d' % basin_nb].isel(time=[0]).item() for basin_nb in range(1,9)]

gdf.head()

In [ ]:
dates = ds_2y.time.astype("datetime64")
dates

In [ ]:
for c,date in enumerate(ds_2y.time.data):
    f = plt.figure(figsize=(12,8))
    ax1 = plt.subplot(121)
    ax1.set_aspect(3)

    vals = (ds_2y.GrIS_total - ds_2y.GrIS_total.isel(time=[0]).item()).data

    ax1.plot(dates,vals,color='k')
    ax1.scatter(dates[c],vals[c],color='k')
    ax1.grid(True)

    ax1.set_ylabel("Gt")
    ax1.set_title("Evolution of Total Greenland Ice Sheet \n%d" %(date.astype('datetime64[Y]').astype(int)+1970))

    ## For Antarctic
    # ax2 = plt.subplot(122, projection=ccrs.SouthPolarStereo())
    ax2 = plt.subplot(122,projection=ccrs.NearsidePerspective(central_longitude=LON_CENTRAL, central_latitude=LAT_CENTRAL))
    ax2.coastlines()                                                                    # add the coastlines
    gl = ax2.gridlines(draw_labels=True)                        # add the longitude / latitude lines
    gl.right_labels = False                                                            # remove latitude labels on the right
    gl.top_labels = False                                                              # remove longitude labels on the top
    ax2.add_feature(cfeature.LAND, zorder=1, edgecolor='k')                             # add land mask
    ax2.set_extent([LON_MIN, LON_MAX, LAT_MIN, LAT_MAX],crs=ccrs.PlateCarree())         # define the extent of the map [lon_min,lon_max,lat_min,lat_max]

    # Plot the sea total precipitation, set the min/max values of the colorbar and the colormap to use
    im = gdf.plot(ax=ax2, column=str(date.astype('datetime64[Y]').astype(int)+1970), transform=ccrs.PlateCarree(), legend=True, cmap='bwr_r',vmin=-800, vmax=800, edgecolor='white', linewidth=0.5)
    # Add the titles and legend
    ax2.set_title("Mass Change of the Greenland Ice Shelf\n since 2003 in $Gt$")

    # Save figure
    plt.savefig('Data/GRACE/figs/evol_anim_%s.png' % str(c).zfill(2))
    print("Figure for {} generated".format(date))                 # print a message when a figure is generated
    plt.close()       

In [ ]:
! pip install imageio
# Import the modules for the animation
import imageio
from pathlib import Path
from IPython.display import Image

# Search image files
image_path = Path('Data/GRACE/figs')
images = sorted(image_path.glob('evol_anim_*.png'))
image_list = []
for file_name in images:
    image_list.append(imageio.imread(file_name))
    
# Save the GIF
imageio.mimwrite('Data/GRACE/figs/animated_antartic.gif',image_list, duration=1000, loop=10)

# Visualisation
with open('Data/GRACE/figs/animated_antartic.gif','rb') as f:
    display(Image(data=f.read(),format='png'))

## 3. Exploration: Correlation with sea Level evolution


The melting of the Greenland Ice Sheet is known to drive the rise of the sea-level, we have also downloaded sea-level data over the same period to see how Ice mass change and sea level correlate.

We are going to use the [Global Ocean Gridded L 4 Sea Surface Heights And Derived Variables Reprocessed 1993 Ongoing](https://www.wekeo.eu/data?view=dataset&dataset=EO%3AMO%3ADAT%3ASEALEVEL_GLO_PHY_L4_MY_008_047) product. This dataset offers altimeter satellite gridded Sea Level Anomalies (SLA) computed with respect to a twenty-year [1993, 2012] mean.


| Parameter | Value |
| :---: | :---|
| **Variables used** | sla |
| **Product identifier** |[EO:MO:DAT:SEALEVEL_GLO_PHY_L4_MY_008_047](https://www.wekeo.eu/data?view=dataset&dataset=EO%3AMO%3ADAT%3ASEALEVEL_GLO_PHY_L4_MY_008_047) |
| **Spatial resolution** | 0.25° x 0.25° |
| **Temporal resolution** | Monthly averaged values |
| **Temporal coverage** | Feburary 1993 May 2023 |


<center>
    <img src="Data/GRACE/sla_wekeo_dataset.png" width="150">
</center>

You can also visit the [Copernicus Marine Service page](https://data.marine.copernicus.eu/product/SEALEVEL_GLO_PHY_L4_MY_008_047/description) dedicated to the product to see more detail about the product and all the different variables that are available.

In [ ]:
ds_s = xr.open_dataset("Data/GRACE/sea_level_Global_two_satellite_199302_202305.nc")
ds_s

The sea level dataset is now almost operationnal ! Contrary to the ice mass dataset, this one contains spatial data, and we can see that it has longitude and latitude coordinates. One last thing we have to do to ease plotting is to convert the longitudes. In the dataset they range from 0° to 360°, whereas the preferred convention in matplotlib is to have longitudes ranging from -180° to 180°.

In [ ]:
ds_s.coords["longitude"] = (ds_s.coords["longitude"]+180) % 360 -180
ds_s = ds_s.sortby(ds_s.longitude)

In [ ]:
date_plot='2018-01'
temp_map = ds_s['sla'].sel(time=date_plot).squeeze()

# Define the caracteristics of the plot
f = plt.figure(figsize=(20, 16))                                                   # create a figure and define its size
ax = plt.axes(projection=ccrs.PlateCarree())                                       # create an ax and select the projection of the map
ax.coastlines()                                                                    # add the coastlines
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)                        # add the longitude / latitude lines
gl.right_labels = False                                                            # remove latitude labels on the right
gl.top_labels = False                                                              # remove longitude labels on the top
ax.add_feature(cfeature.LAND, zorder=1, edgecolor='k')                             # add land mask
ax.set_extent([-180, 180, -90, 90],crs=ccrs.PlateCarree())                             # define the extent of the map [lon_min,lon_max,lat_min,lat_max]

# Plot the sea surface temperature, set the min/max values of the colorbar and the colormap to use
im = ax.pcolor(temp_map['longitude'].data, temp_map['latitude'].data, temp_map, vmin=1, vmax=-1, cmap='BrBG')
    

# Add the titles and legend
ax.set_title('Sea Surface Level anomaly on {}\n compared to 1993-2012 mean'.format(date_plot),fontsize=25)              # add a title to the figure
cbar = f.colorbar(im,ax=ax,fraction=0.02, pad=0.02)                                            # add the colorbar
cbar.set_label('m', rotation=270,fontsize=25, labelpad=25)                                    # add the title of the colorbar

It is possible to subset a spatial dataset just like a timeseries dataset. The only difference is that this time, instead of the *time* coordinate, the selection constraints are applied to the *longitude* and/or *latitude* coordinates. Here is an example with the `slice` method.

In [ ]:
# Selecting only the Southern latitudes : 
# selecting only the years 2003 to 2020, for consistency with ice mass dataset
ds_s_north = ds_s.sel(latitude=slice(50, 80)).sel(time=slice('2003', '2022'))
ds_s_north

In [ ]:
# Correlation with Ice mass loss

# yearly values
ds_s_north_y = ds_s_north.resample(time='1Y').mean()
ds_s_north_y

Let's now calculate the correlation between the sea-level and the total Antarctic Ice-mass balance. We are going to use the **Pearson Coefficient**. It determines the linear correlation between two datasets. It varies between -1 and 1. A value of 1 implies that there is a perfectly linear relationship between the two distributions, and a value of -1 indicates a perfect inverse linear relationship. A value close to zero indicates that there is no linear pattern between the two datasets.

In [ ]:
# to ease  calculation: manually specifying years
time = np.arange(2003, 2023)
vals_ice = ds_y['GrIS_total'].data
vals_sea = ds_s_north_y.mean(dim=['latitude', 'longitude'])['sla'].data

vals_sea.shape

In [ ]:
fig, ax = plt.subplots()
ax.scatter(vals_ice, vals_sea)
ax.grid(True)
ax.set_title("Scatter plot of SLA vs Ice mass change over 2003-2023.")
ax.set_xlabel("AIS mass change (GT)")
ax.set_ylabel("SLA (m)")

# correlation calculation using scipy
from scipy.stats import pearsonr
corr, _ = pearsonr(vals_ice, vals_sea)

print('Pearsons correlation: %.3f' % corr)

The Pearson coefficient of -0.793 indicates that there is a fairly strong inverse correlation between sea level and ice mass change. This makes sense, because as the ice melts (Ice mass change decreasing), the sea level increases. We can also see on the scatter plot that the points corresponding to the highest ice masses correspond to the lowest sea levels, and vice versa.

However, how is the sea level spatially distributed ? Let's emulate the basins from the Antarctic dataset by creating several sectors over the Antarctic ocean. We will create regular sectors with a 20° longitudinal length, covering from -90° to -60° latitude

For each sector, the sea levels will be spatially averaged, and the yearly trend computed.

In [ ]:
# creating a dataframe with regular sectors:
nb_sectors = 36
lon_slices = np.linspace(-180, 180, nb_sectors+1)
poly_list = []
y_series_list = []
trend_list = []
centroid_list = []


for i in range(nb_sectors):
    poly_list.append(Polygon([[lon_slices[i],50], [lon_slices[i+1],50], [lon_slices[i+1],80], [lon_slices[i],80]]))
    centroid_list.append((0.5*(lon_slices[i]+lon_slices[i+1]), 65)) # centroid for annotation
    
    tmp = ds_s_north_y.sel(longitude=slice(lon_slices[i], lon_slices[i+1]))
    ts = tmp.mean(dim=['latitude', 'longitude'])['sla']    # spatially averaging sea level over the sector
    y_series_list.append(ts.data)
    
    slope, intercept = np.polyfit(np.arange(len(ts.data)).reshape(-1,1).flatten(), ts.data, 1)   # computing yearly sea level trend
    trend_list.append(slope*1000)    # conversion to mm/year
    
    

gdf_sea = gpd.GeoDataFrame(index=range(nb_sectors), crs='epsg:3857', geometry=poly_list)
gdf_sea['sea_level_timeseries'] = y_series_list
gdf_sea['linear_trend_sea_level'] = trend_list
gdf_sea['sector_nb'] = range(1, nb_sectors+1)

gdf_sea['coords'] = centroid_list

gdf_sea

In [ ]:
# Define the caracteristics of the plot
f = plt.figure(figsize=(7, 7))                                                    # create a figure and define its size
## Antarctic usage
ax = plt.axes(projection=ccrs.NorthPolarStereo())                                       # create an ax and select the projection of the map
# ax = plt.axes(projection=ccrs.NearsidePerspective(central_longitude=LON_CENTRAL, central_latitude=LAT_CENTRAL))
ax.coastlines()                                                                    # add the coastlines
gl = ax.gridlines(draw_labels=True)                        # add the longitude / latitude lines
gl.right_labels = False                                                            # remove latitude labels on the right
gl.top_labels = False                                                              # remove longitude labels on the top
ax.add_feature(cfeature.LAND, zorder=2, edgecolor='k')                             # add land mask
# ax.set_extent([LON_MIN, LON_MAX, LAT_MIN, LAT_MAX],crs=ccrs.PlateCarree())         # define the extent of the map [lon_min,lon_max,lat_min,lat_max]
# ax.set_extent([LON_MIN, LON_MAX, LAT_MIN, LAT_MAX],crs=ccrs.PlateCarree())

# Plot the sea total precipitation, set the min/max values of the colorbar and the colormap to use
im = gdf_sea.plot(ax=ax, column='linear_trend_sea_level', transform=ccrs.PlateCarree(), cmap='BrBG', vmin=-3, vmax=3, edgecolor='k', linewidth=0.5, legend=True)
# Add the titles and legend
ax.set_title("Sea Level trends around the Greenland \n between 2003-2023 in $mm.y^{-1}$")

# ## you can add this anotation over Antarctic
# for idx, row in gdf_sea.iterrows():
#     plt.annotate(text=row['sector_nb'], xy=row['coords'],
#                  horizontalalignment='center', transform=ccrs.PlateCarree(), color='b', fontsize=8)
    

We can see that the sea levels also have strong spatial variability! Barents Sea and Fram Strait has a rising trend going up to 3 mm/year, but near West Antarctica!

Areas with decreasing sea levels remain few and sparse (see below). On average, the global sea level is going up about **2 millimeters per year**. Monitoring the waters fluxes, from glaciers to ocean, from ocean to rain, and so on, remains crucial to understand climate change better, improve the predictions and develop adequate mitigation solutions.


# Any take home message?